# Goals
The aim is to recreate FE versions of the test problems form the BICGStab paper and export them as mtx files.

## Set Up

In [52]:
using Gridap

n = 8; Omega = (0.0,1.0,0.0,1.0);
Grid = CartesianDiscreteModel(Omega, (n,n))
# Grid has 4 corners and 4 edges
# "boundary" is all elements of the boundary of Omega
# Tags 1-4 are the corners and Tags 5-8 are the edges.
# Label each of the six sides differently
# boundary
Tags = get_face_labeling(Grid)
add_tag_from_tags!(Tags,"y=0",[1,2,5])
add_tag_from_tags!(Tags,"OtherEdges",[3,4,6,7,8])
# Piecewise linear Lagrange elements
order = 1; Elements = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(Grid,Elements;conformity=:H1,dirichlet_tags="y=0")
# Incorporating Homogeneous Dirichlet boundary conditions.
g(x) = 0.0
U = TrialFESpace(V,g)
# Triangulating and defining measures
degree = 2
Ω = Triangulation(Grid)
dΩ = Measure(Ω,degree)
# Setting up boundary triangulation/measure for Neumann conditions 
Γ = BoundaryTriangulation(Grid,tags="OtherEdges")
dΓ = Measure(Γ,degree)
# Setting up a simple test problem
f(x) = 1.0; h(x) = 3.0
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ
b(v) = ∫( v*f )*dΩ + ∫( v*h )*dΓ
# Set up FE operator
op = AffineFEOperator(a,b,U,V)
# Set up assembler
# Define Assembler
assem = SparseMatrixAssembler(V,U)

GenericSparseMatrixAssembler()

In [45]:
fieldnames(AffineFEOperator)

(:trial, :test, :op)

In [51]:
op.test

UnconstrainedFESpace()

In [29]:
# Piecewise linear Lagrange elements
order = 1; Elements = ReferenceFE(lagrangian,Float64,order)
V0 = TestFESpace(Grid,Elements;conformity=:H1,dirichlet_tags="boundary")

UnconstrainedFESpace()

In [10]:
# Construct the discrete model
model = CartesianDiscreteModel(domain=(0.0,1.0,0.0,1.0), partition=(4,4))

# Construct the FEspace
order = 1
diritag = "boundary"
fespace = ConformingFESpace(Float64,model,order,diritag)

# Define test and trial spaces
V = TestFESpace(fespace)
U = TrialFESpace(fespace,ufun)

# Define integration mesh and quadrature
trian = Triangulation(model)
quad = CellQuadrature(trian,order=2)

# Define the source term
bfield = CellField(trian,bfun)

# Define forms of the problem
a(v,u) = inner(∇(v), ∇(u))
b(v) = inner(v,bfield)

# Define Assembler
assem = SparseMatrixAssembler(V,U)

# Define the FEOperator
op = LinearFEOperator(a,b,V,U,assem,trian,quad)
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

LoadError: MethodError: no method matching Gridap.Geometry.CartesianDescriptor(; domain=(0.0, 1.0, 0.0, 1.0), partition=(4, 4))
[0mClosest candidates are:
[0m  Gridap.Geometry.CartesianDescriptor([91m::VectorValue{D, T} where T[39m, [91m::VectorValue{D, T} where T[39m, [91m::Any[39m; map, isperiodic) where D at C:\Users\struther\.julia\packages\Gridap\oABgW\src\Geometry\CartesianGrids.jl:82[91m got unsupported keyword arguments "domain", "partition"[39m
[0m  Gridap.Geometry.CartesianDescriptor([91m::VectorValue{D, T} where T[39m, [91m::VectorValue{D, T} where T[39m, [91m::Any[39m, [91m::Function[39m) where D at deprecated.jl:70[91m got unsupported keyword arguments "domain", "partition"[39m
[0m  Gridap.Geometry.CartesianDescriptor([91m::VectorValue{D, T} where T[39m, [91m::Tuple{Vararg{T, D}} where T[39m, [91m::Any[39m; map, isperiodic) where D at C:\Users\struther\.julia\packages\Gridap\oABgW\src\Geometry\CartesianGrids.jl:30[91m got unsupported keyword ar

In [8]:
import Pkg; Pkg.add("Gridap")

   Resolving package versions...
   Installed ArgCheck ──────────── v2.3.0
   Installed Distances ─────────── v0.10.7
   Installed NLSolversBase ─────── v7.8.3
   Installed ArrayLayouts ──────── v0.8.14
   Installed FastGaussQuadrature ─ v0.4.9
   Installed LineSearches ──────── v7.1.1
   Installed Combinatorics ─────── v1.0.2
   Installed UnPack ────────────── v1.0.2
   Installed Parameters ────────── v0.12.3
   Installed NLsolve ───────────── v4.5.1
   Installed BlockArrays ───────── v0.16.21
   Installed LightXML ──────────── v0.9.0
   Installed Gridap ────────────── v0.17.15
   Installed WriteVTK ──────────── v1.16.0
   Installed SparseMatricesCSR ─── v0.6.6
   Installed NearestNeighbors ──── v0.4.12
   Installed BSON ──────────────── v0.3.6
   Installed PolynomialBases ───── v0.4.14
   Installed AbstractTrees ─────── v0.4.3
   Installed JLD2 ──────────────── v0.4.28
    Updating `C:\Users\struther\.julia\environments\v1.6\Project.toml`
  [56d4f2e9] + Gridap v0.17.15
    Updating `

In [6]:
using Gridap

LoadError: ArgumentError: Package Gridap not found in current path:
- Run `import Pkg; Pkg.add("Gridap")` to install the Gridap package.
